In [ ]:
import ROOT
from T2KStyle.T2KStyle import GetT2K
%jsroot on
style = GetT2K(3)
ROOT.gROOT.SetStyle("T2K")

c = ROOT.TCanvas('c', '', 800, 600)
cc = ROOT.TCanvas('cc', '', 800, 600)
ccc = ROOT.TCanvas('ccc', '', 800, 600)
cl = ROOT.TCanvas('cl', '', 1600, 1200)
cw = ROOT.TCanvas('c2', '', 1600, 600)
cw3 = ROOT.TCanvas('c3', '', 2400, 600)

In [ ]:
path = '~/DATA/'

file_mc = [ROOT.TFile(path + '/hs5_iter0.root')]
file_data = [ROOT.TFile('~/DATA/DESY_2020/s_412_430_02T_iter9.root', 'READ')]

In [ ]:
name = ['0 mm', '1 mm', '2 mm', '3 mm', '4 mm', '5 mm']
color = [ROOT.kRed, ROOT.kViolet, ROOT.kBlue, ROOT.kBlack, ROOT.kGreen, ROOT.kMagenta]
tol = 0.0005
cut = [f'multiplicity > 1 && abs(dx[][0] - 0.000) < {tol}',
       f'multiplicity > 1 && abs(dx[][0] - 0.001) < {tol}',
       f'multiplicity > 1 && abs(dx[][0] - 0.002) < {tol}',
       f'multiplicity > 1 && abs(dx[][0] - 0.003) < {tol}',
       f'multiplicity > 1 && abs(dx[][0] - 0.004) < {tol}',
       f'multiplicity > 1 && abs(dx[][0] - 0.005) < {tol}'
      ]

Absolute charge

In [ ]:
var = ['pad_charge[][0]', 'pad_charge[][1]', 'pad_charge[][2]']

title = ['Q_{1}', 'Q_{2}', 'Q_{3}']

bins = [200, 200, 200]
bin_s = [0., 0, 0]
bin_e = [4000, 1500, 400]

Charge ratio

In [ ]:
var = ['pad_charge[][0]', 'pad_charge[][1] / pad_charge[][0]', 'pad_charge[][2] / pad_charge[][0]']

title = ['Q_{1}', 'Q_{2}/Q_{1}', 'Q_{3}/Q_{1}']

bins = [200, 210, 200]
bin_s = [0., 0, 0]
bin_e = [4000, 1.1, 0.3]

Time

In [ ]:
var = ['pad_charge[][0]', 'pad_time[][1] - pad_time[][0]', 'pad_time[][2] - pad_time[][0]']

title = ['Q_{1}', 't_{2} - t_{1}', 't_{3} - t_{1}']

bins = [200, 30, 50]
bin_s = [0., 0, 0]
bin_e = [4000, 30, 50]

FWHM

In [ ]:
var = ['wf_fwhm[][0]', 'wf_fwhm[][1]', 'wf_fwhm[][2]']

title = ['FWHM WF leading', 'FWHM WF second', 'FWHM WF third']

bins = [30, 60, 100]
bin_s = [0., 0, 0]
bin_e = [30, 60, 100]

# Draw

In [ ]:
cw3.Clear()
cw3.Divide(3)

h = []
hs = []
ev_prev = -1
hs.append(ROOT.THStack('hsl', ''))
hs.append(ROOT.THStack('hss', ''))
hs.append(ROOT.THStack('hst', ''))

ls = [0, 1, 2, 3, 4, 5]
opt = ['l', 's', 't']
    
# working on MC
RC = '55'
for o_num, o in enumerate(opt):
    for i in ls:
        print(f'MC {o} {i}\r', end='')
        h.append(ROOT.TH1F(f'h_mc_{o}_{name[i]}_RC{RC}', '', bins[o_num], bin_s[o_num], bin_e[o_num]))
        file_mc[0].outtree.Project(f'h_mc_{o}_{name[i]}_RC{RC}', var[o_num], cut[i])
        h[-1].SetLineStyle(2)
        h[-1].SetLineColor(color[i])
        h[-1].Scale(1./h[-1].Integral())
        hs[o_num].Add(h[-1])
        
# working on DATA
for o_num, o in enumerate(opt):
    for i in ls:
        print(f'DATA {o} {i}\r', end='')
        h.append(ROOT.TH1F(f'h_data_{o}_{name[i]}', '', bins[o_num], bin_s[o_num], bin_e[o_num]))
        file_data[0].outtree.Project(f'h_data_{o}_{name[i]}', var[o_num], cut[i])
        h[-1].SetLineColor(color[i])
        h[-1].Scale(1./h[-1].Integral())
        hs[o_num].Add(h[-1])

cw3.cd(1)
hs[0].Draw('histo nostack')
hs[0].GetXaxis().SetTitle(title[0])
hs[0].SetTitle(f'Charge in the leading')
ROOT.gPad.BuildLegend()

cw3.cd(2)
hs[1].Draw('histo nostack')
hs[1].GetXaxis().SetTitle(title[1])
hs[1].SetTitle(f'Charge spreading')
ROOT.gPad.BuildLegend()

cw3.cd(3)
hs[2].Draw('histo nostack')
hs[2].GetXaxis().SetTitle(title[2])
hs[2].SetTitle(f'Charge spreading')
ROOT.gPad.BuildLegend()

cw3.Draw() 

In [ ]:
c.cd()
mg = ROOT.TMultiGraph()
mg_r = ROOT.TMultiGraph()
mg_r2 = ROOT.TMultiGraph()

x = [0, 1, 2, 3, 4, 5]

def getVal(h):
#     return h.GetMean()
    h.Fit('landau', 'Q')
    h.Fit('landau', 'Q')
    return h.GetFunction('landau').GetParameter(1)

def getValErr(h):
#     return h.GetMeanError()
    h.Fit('landau', 'Q')
    return h.GetFunction('landau').GetParError(1)

def set_gr(gr, col, marker):
    gr.SetMarkerStyle(marker)
    gr.SetLineColor(col)
    gr.SetMarkerColor(col)
    
# RC = ['50', '55', '60']
RC = ['55']
col = [ROOT.kBlue, ROOT.kRed, ROOT.kGreen]

gr = []
gr_r = []
gr_r2 = []

gr.append(ROOT.TGraphErrors())
set_gr(gr[-1], ROOT.kBlack, 22)
num = 0
for i in range(len(h)):
    parse = h[i].GetName().split('_')
    if num >= len(name) or parse[1] != 'data' or parse[3] != name[num] or parse[2] != 'l': continue
    gr[-1].SetPoint(gr[-1].GetN(), x[num], getVal(h[i]))
    gr[-1].SetPointError(gr[-1].GetN()-1, 0, getValErr(h[i]))
    num += 1
    
gr.append(ROOT.TGraphErrors())
set_gr(gr[-1], ROOT.kBlack, 20)
num = 0
for i in range(len(h)):
    parse = h[i].GetName().split('_')
    if num >= len(name) or parse[1] != 'data' or parse[3] != name[num] or parse[2] != 's': continue
    gr[-1].SetPoint(gr[-1].GetN(), -x[num] + 10.19, getVal(h[i]))
    gr[-1].SetPointError(gr[-1].GetN()-1, 0, getValErr(h[i]))
    num += 1
    
gr.append(ROOT.TGraphErrors())
set_gr(gr[-1], ROOT.kBlack, 20)
num = 0
for i in range(len(h)):
    parse = h[i].GetName().split('_')
    if num >= len(name) or parse[1] != 'data' or parse[3] != name[num] or parse[2] != 't': continue
    gr[-1].SetPoint(gr[-1].GetN(), x[num] + 10.19, getVal(h[i]))
    gr[-1].SetPointError(gr[-1].GetN()-1, 0, getValErr(h[i]))
    num += 1

data_id = len(gr) - 1

for num_rc, RC_val in enumerate(RC):
    gr.append(ROOT.TGraphErrors())
    gr_r.append(ROOT.TGraphErrors())
    set_gr(gr[-1], col[num_rc], 22)
    set_gr(gr_r[-1], col[num_rc], 22)
    num_s = 0
    num_t = 0
    num_l = 0
    for i in range(len(h)):
        parse = h[i].GetName().split('_')
        if num_l < len(name) and parse[1] == 'mc' and parse[3] == name[num_l] and parse[2] == 'l' and parse[4] == 'RC' + RC_val: 
            gr[-1].SetPoint(gr[-1].GetN(), x[num_l], getVal(h[i]))
            gr[-1].SetPointError(gr[-1].GetN()-1, 0, getValErr(h[i]))

            data_point = x.index(num_l)
            ratio = getVal(h[i]) / gr[data_id-2].GetY()[data_point]
            un_a = getValErr(h[i]) / getVal(h[i])
            un_b = gr[data_id-2].GetErrorY(data_point) / gr[data_id-2].GetY()[data_point]
            gr_r[-1].SetPoint(gr_r[-1].GetN(), x[num_l], ratio)
            gr_r[-1].SetPointError(gr_r[-1].GetN()-1, 0, ratio*((un_a*un_a + un_b*un_b)**0.5))
            num_l += 1
        if num_s < len(name) and parse[1] == 'mc' and parse[3] == name[num_s] and parse[2] == 's' and parse[4] == 'RC' + RC_val: 
            gr[-1].SetPoint(gr[-1].GetN(), -x[num_s] + 10.19, getVal(h[i]))
            gr[-1].SetPointError(gr[-1].GetN()-1, 0, getValErr(h[i]))

            data_point = x.index(num_s)
            ratio = getVal(h[i]) / gr[data_id-1].GetY()[data_point]
            un_a = getValErr(h[i]) / getVal(h[i])
            un_b = gr[data_id-1].GetErrorY(data_point) / gr[data_id-1].GetY()[data_point]
            gr_r[-1].SetPoint(gr_r[-1].GetN(), -x[num_s] + 10.19, ratio)
            gr_r[-1].SetPointError(gr_r[-1].GetN()-1, 0, ratio*((un_a*un_a + un_b*un_b)**0.5))
            num_s += 1
        if num_t < len(name) and parse[1] == 'mc' and parse[3] == name[num_t] and parse[2] == 't' and parse[4] == 'RC' + RC_val: 
            gr[-1].SetPoint(gr[-1].GetN(), x[num_t] + 10.19, getVal(h[i]))
            gr[-1].SetPointError(gr[-1].GetN()-1, 0, getValErr(h[i]))

            data_point = x.index(num_t)
            ratio = getVal(h[i]) / gr[data_id].GetY()[data_point]
            un_a = getValErr(h[i]) / getVal(h[i])
            un_b = gr[data_id].GetErrorY(data_point) / gr[data_id].GetY()[data_point]
            gr_r[-1].SetPoint(gr_r[-1].GetN(), x[num_t] + 10.19, ratio)
            gr_r[-1].SetPointError(gr_r[-1].GetN()-1, 0, ratio*((un_a*un_a + un_b*un_b)**0.5))
            num_t += 1
        
for g in gr:
    mg.Add(g)
for g in gr_r:
    mg_r.Add(g)

cw.cd()
cw.Clear()
cw.Divide(2)
cw.cd(1)
mg.Draw('ap')
mg.SetTitle(';x_{track}-x_{pad} [mm];Mean ')

cw.cd(2)
mg_r.Draw('ap')
mg_r.SetTitle(';x_{track}-x_{pad} [mm];MC / DATA')

cw.Draw()